###Imports

In [6]:
import numpy as np
import cv2
import csv
import glob
import os
import pandas as pd
import time

from IPython.display import clear_output

def update_progress(progress):
    bar_length = 60
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

### **Preprocessing**
##Iterate over frames in each video to:
1. Crop the face using haarcascade face detector.
2. Calculate flow between each two consecutive frames
3. Save the cropped image and the flow image.

In [3]:
# Uncomment following when running on local device

# Add Home dir upto BAUM folder
home_dir = '..\\'#TODO add you home directory
baum_dir = home_dir + 'BAUM\\'

In [4]:
src_folder_name = 'BAUM1a_MP4_all'
dst_folder_spatial = 'imgs_spatial_1a'
dst_folder_flow = 'imgs_flow_1a'
src_csv_name = "Annotations_BAUM1a.xlsx"
target_codes = [1, 3, 4, 5, 7, 8]


In [5]:
temp_image = np.array([])
none_image = None

data_labels_xl =  pd.read_excel(baum_dir + src_csv_name)

data_labels = np.array(data_labels_xl)

videos = glob.glob(baum_dir + src_folder_name + '/*/*.mp4')
videos = sorted(videos)
print(len(videos), " videos was detected in baum_dir")


# Create a face detec tion module 
calssifiers =['haarcascade_frontalface_default.xml',
              'haarcascade_frontalface_alt2.xml',
              'haarcascade_frontalcatface_extended.xml',
              'haarcascade_frontalcatface.xml',
              'haarcascade_frontalface_alt_tree.xml',
              'haarcascade_frontalface_alt.xml']

def detectFace(img):
    for classifier in calssifiers:
        face_cascade = cv2.CascadeClassifier(home_dir +'face_detectors/' + classifier)
        faces = face_cascade.detectMultiScale(img, 1.3, 1)
        if (len(faces) > 0):
            return faces
    faces = []
    return faces

274  videos was detected in baum_dir


In [14]:
data_generated = 0
cnt = 0
if not data_generated:
  videoSkipped = 0
  allbadFrames_count = 0
  videoProcessed = 0
  for video in videos:
      #print(video)
      # Test if this video has been already procedded
      already_processed = 0
      flow_path = video.replace(src_folder_name, dst_folder_flow).replace('.mp4', '\\')
      print (flow_path)
      update_progress(float(cnt)/len(videos))
      cnt += 1
      spatial_path = video.replace(src_folder_name, dst_folder_spatial).replace('.mp4', '\\')
      flow_relative_path = flow_path.split('\\')[-4] + '\\' + flow_path.split('\\')[-3] + '\\' + flow_path.split('\\')[-2] 
      spatial_relative_path = spatial_path.split('\\')[-4] + '\\' + spatial_path.split('\\')[-3] + '\\' + spatial_path.split('\\')[-2]
      folder = spatial_relative_path
      #print(folder)
      contents = glob.glob(folder + "\\" + "*.png")
      data_count = len(contents)
      #print(data_count)
      if  data_count > 15:
          already_processed = 1

      # Test if this video in one of the 6 classes
      sample_name = video.split('\\')[-1].split('.')[0]
      label_found = 0
      for label in data_labels:
        if label[3] == sample_name:
            emotion = label[4]
            code = label[5]
            if code in target_codes:
              label_found = 1
              #print("In labels")
              break

      if (not label_found) or (already_processed):
        pass #print("already processed or out of classes")
      else:
        #print(video)
        videoProcessed += 1
        cap = cv2.VideoCapture(video)
        no_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
        ret, old_frame = cap.read()
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

        faces = detectFace(old_gray)
        count = 0
        while (len(faces) == 0):    
            if count > no_of_frames - 1: #Stop looking for a valid frame at the end of the video #
                break
            count+=1
            #print("First Frame is not good for face detection ", count, no_of_frames)
            ret, old_frame = cap.read()
            if isinstance(old_frame, type(temp_image)) and not isinstance(old_frame, type(none_image)) :
                old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
                faces = detectFace(old_gray)
            else:
                #print ("bad frame", type(old_frame), video)
                pass 
                
                
                      
        if (len(faces) == 0):
            #print ("All Video Frames are BAD", video)
            videoSkipped += 1
            continue
        
        (x,y,w,h) = faces[0]
        old_gray = old_gray[y:y+h, x:x+w]
        old_color = old_frame[y:y+h, x:x+w]
        size_gray = old_gray.shape
        size_color = old_color.shape

        # Create a mask image for drawing purposes
        mask = np.zeros_like(old_color)
        hsv = np.zeros_like(old_color)
        hsv[...,1] = 255
        
        noFacesCnt = 0
        calcOpticalFlow = 0
        for i in range(no_of_frames - 1):
            ret,frame = cap.read()
            if isinstance(frame, type(temp_image)) and not isinstance(frame, type(none_image)) :
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                faces = detectFace(frame_gray)
                if len(faces) == 0:
                    calcOpticalFlow = 0
                    noFacesCnt += 1
                    allbadFrames_count +=1
                    #print ("No Faces Found ", noFacesCnt, no_of_frames, video)
                    continue
                
                (x,y,w,h)  = faces[0]
                frame_gray = frame_gray[y:y+h, x:x+w]
                roi_color = frame[y:y+h, x:x+w]
                
                if(frame_gray.shape[0] < 90 or roi_color.shape[0] < 90 ):
                    calcOpticalFlow = 0
                    allbadFrames_count +=1
                    #print("Skip this frame , small image!! ", video , i)
                    continue
                                
                #print(frame_gray.shape)
                frame_gray = cv2.resize(frame_gray, (size_gray[1],size_gray[0]))
                roi_color = cv2.resize(roi_color, (size_color[1],size_color[0]))
                #print(frame_gray.shape, "after")
              
                calcOpticalFlow += 1
                            
                if (calcOpticalFlow > 1):
                    flow = cv2.calcOpticalFlowFarneback(old_gray,frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)            
                    mag, ang   = cv2.cartToPolar(flow[...,0], flow[...,1])
                    hsv[...,0] = ang*180/np.pi/2
                    hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
                    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
                    
                    str_i = '{:03}'.format(i) # max 999 frames

                    flow_relative_name = flow_relative_path + '\\' + str_i + '.png'
                    spatial_relative_name = spatial_relative_path + '\\' + str_i + '.png'
                    
                    flow_name_abs = baum_dir + flow_relative_name
                    spatial_name_abs = baum_dir + spatial_relative_name

                    flow_folder = flow_name_abs.replace('\\'+str_i+'.png', '')
                    spatial_folder = spatial_name_abs.replace('\\'+str_i+'.png', '')

                    if not os.path.isdir(flow_folder):
                        os.makedirs(flow_folder)
                    if not os.path.isdir(spatial_folder):
                        #print("create folder: ", spatial_folder)
                        os.makedirs(spatial_folder)
                    
                    cv2.imwrite(flow_name_abs, rgb)
                    cv2.imwrite(spatial_name_abs,roi_color)
   
                # Now update the previous frame and previous points
                old_gray = frame_gray.copy()
              
  print("No of skipped videos is: ", videoSkipped)
  print("No of allbadFrames is: ", allbadFrames_count)
  print("No of video processed is ", videoProcessed)

Progress: [############################################################] 99.6%
..\BAUM\imgs_flow_1a\s031\S031_010\
No of skipped videos is:  0
No of allbadFrames is:  233
No of video processed is  173


# Get All Written Images

In [15]:
from glob import glob
import os, time
import pandas as pd


csv_file_name = 'data_6_classes_1a.csv'


data_labels_xl =  pd.read_excel(baum_dir + src_csv_name)

csv_file_name = baum_dir + csv_file_name
data_frame = pd.DataFrame([["flow", "spatial",  "emotion", "code"]])
data_frame.to_csv(csv_file_name, mode='w', header=False, index = False, sep = ',')

print ("Target scv file is: ", csv_file_name)

flow = baum_dir + dst_folder_flow +'\\'
spatial = baum_dir + dst_folder_spatial + '\\'

flowImgs = glob(flow+'*/*/*.png')
count = 0
for img in flowImgs:
    flowImg = img
    spatialImg = img.replace(dst_folder_flow, dst_folder_spatial)
    if (os.path.isfile(spatialImg)):
        count += 1
        vidName = img.split('\\')[-2]
        vidLabel = str(data_labels_xl.loc[data_labels_xl['Clip Name'] == vidName]['Emotion'].values[0])
        vidCode = str(data_labels_xl.loc[data_labels_xl['Clip Name'] == vidName]['Emotion Code'].values[0])
        if (int(vidCode) in target_codes):
            #print ('Needed Emotion')
            #print (flowImg, spatialImg, vidLabel, vidCode)
            data_frame =  pd.DataFrame([[flowImg, spatialImg,  vidLabel, vidCode]])
            data_frame.to_csv(csv_file_name, mode='a', header=False, index = False, sep = ',')

print ("Total found images: ", len(flowImgs), count)

Target scv file is:  ..\BAUM\data_6_classes_1a.csv
Total found images:  18235 18235


### **Splitting data to train / valid**

In [16]:
import pandas as pd
import numpy as np
baum_dir = home_dir + 'BAUM\\'

csv_file = 'data_6_classes_1a.csv'
data = pd.read_csv(baum_dir + csv_file )
data.sort_values(by=['code'])
data = np.array(data)

training_data_file = baum_dir + "training_data_1a.csv"
validation_data_file = baum_dir + "validation_data_1a.csv"
target_codes = [1, 3, 4, 5, 7, 8]
#anger , disgust , Fear , Happiness, Sadness, Surprise

new_target_codes = [1, 2, 3, 4, 5, 6]


data_frame = pd.DataFrame([["flow", "spatial",  "emotion", "code"]])
data_frame.to_csv(training_data_file, mode='w', header=False, index = False, sep = ',')
data_frame.to_csv(validation_data_file, mode='w', header=False, index = False, sep = ',')

def write_to_file(seg, valid): 
    data_frame =  pd.DataFrame([[seg[0].replace(baum_dir, ''), seg[1].replace(baum_dir, ''),  seg[2], seg[3]]]) #convert absolute path to relative path 
    if valid:
        data_frame.to_csv(validation_data_file, mode='a', header=False, index = False, sep = ',')
    else: 
        data_frame.to_csv(training_data_file, mode='a', header=False, index = False, sep = ',')
  
val = 8 # 7 train to 1 valid
last_folder = ""
count = 0
for row in data:
    file_name = row[0]
    #print(file_name)
    count += 1
    update_progress(float(count)/len(flowImgs))
    folder = file_name.split('\\')[-3]
    row[3] = new_target_codes[target_codes.index(row[3])]
    frame = [row[0], row[1], row[2], row[3]]
    if ( folder != last_folder ):
        val -= 1
        if val == 0:
            valid = 1
            write_to_file(frame, 1)
            val = 8
        else:
            valid = 0
            write_to_file(frame, 0)
    elif valid:
        write_to_file(frame, 1)
    elif not valid:
        write_to_file(frame, 0)
    last_folder = folder
    
print ("Done", "\nTraining File: ",training_data_file , "\nValidation File: ", validation_data_file)

Progress: [############################################################] 100.0%
Done 
Training File:  ..\BAUM\training_data_1a.csv 
Validation File:  ..\BAUM\validation_data_1a.csv


## Concatenate multiple datasets
BAUM-1a, BAUM1s

In [17]:
import shutil
import glob


#import csv files from folder
allFiles_train = ['training_data_1s.csv', 'training_data_1a.csv']
allFiles_valid = ['validation_data_1s.csv', 'validation_data_1a.csv']


with open(baum_dir + 'training_data_mixed.csv', 'wb') as outfile_train:
    for i, fname in enumerate(allFiles_train):
        update_progress(float(i)/len(flowImgs))
        with open(baum_dir + fname, 'rb') as infile:
            if i != 0:
                infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
            shutil.copyfileobj(infile, outfile_train)
            print(fname + " has been imported.")
            
with open(baum_dir + 'validation_data_mixed.csv', 'wb') as outfile_valid:
    for i, fname in enumerate(allFiles_valid):
        update_progress(float(i)/len(flowImgs))
        with open(baum_dir + fname, 'rb') as infile:
            if i != 0:
                infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
            shutil.copyfileobj(infile, outfile_valid)
            print(fname + " has been imported.")

Progress: [------------------------------------------------------------] 0.0%
validation_data_1a.csv has been imported.
